In [ ]:
"""
BIBLIOTECA DE FILMES
Funcionalidades:
1. Cadastrar livros
2. Listar todos os livros
3. Buscar livro por título
4. Gerar gráfico de livros
5. Sair
"""



import matplotlib.pyplot as plt
import json
import os
from datetime import datetime
!pip install pyinstaller

ARQUIVO_DADOS = "biblioteca.json"

def salvar_dados():
    """Salva os dados das listas globais em um arquivo JSON."""
    dados = {
        "biblioteca": [vars(livro) for livro in biblioteca],
        "usuarios": [vars(usuario) for usuario in usuarios],
        "emprestimos": emprestimos
    }
    with open(ARQUIVO_DADOS, 'w', encoding='utf-8') as f:
        json.dump(dados, f, indent=2)
    print(f"✅ Dados salvos em {ARQUIVO_DADOS}!")

def carregar_dados():
    """Carrega os dados do arquivo JSON para as listas globais."""
    if not os.path.exists(ARQUIVO_DADOS):
        return  # Se o arquivo não existe, ignora

    with open(ARQUIVO_DADOS, 'r', encoding='utf-8') as f:
        dados = json.load(f)

        # Preenche a lista 'biblioteca' com objetos Livro
        for livro_data in dados["biblioteca"]:
            livro = Livro(
                titulo=livro_data["titulo"],
                autor=livro_data["autor"],
                genero=livro_data["genero"],
                ano=livro_data["ano"],
                copias=livro_data["copias_disponiveis"]
            )
            livro.copias_emprestadas = livro_data["copias_emprestadas"]
            biblioteca.append(livro)

        # Preenche a lista 'usuarios' com objetos Usuario
        for usuario_data in dados["usuarios"]:
            usuario = Usuario(
                nome=usuario_data["nome"],
                id=usuario_data["id"],
                contato=usuario_data["contato"]
            )
            usuario.livros_emprestados = usuario_data["livros_emprestados"]
            usuarios.append(usuario)

        # Preenche a lista 'emprestimos'
        emprestimos.extend(dados["emprestimos"])
    print(f"✅ Dados carregados de {ARQUIVO_DADOS}!")


# CLASSE PARA LIVROS
class Livro:
    def __init__(self, titulo, autor, genero, ano, copias):
        self.titulo = titulo
        self.autor = autor
        self.genero = genero
        self.ano = ano
        self.copias_disponiveis = copias
        self.copias_emprestadas = 0

    def mostrar_info(self):
        return f"Título: {self.titulo} | Autor: {self.autor} | Gênero: {self.genero} | Ano: {self.ano} | Disponíveis: {self.copias_disponiveis}"

# CLASSE PARA USUÁRIOS

class Usuario:
    def __init__(self, nome, id, contato):
        self.nome = nome
        self.id = id
        self.contato = contato
        self.livros_emprestados = []

    def mostrar_info(self):
        return f"Nome: {self.nome} | ID: {self.id} | Contato: {self.contato}"

# LISTAS GLOBAIS (SIMULAM "BANCO DE DADOS")
biblioteca = []
usuarios = []
emprestimos = []

# FUNÇÕES PRINCIPAIS

# CADASTRAR LIVROS
def cadastrar_livro():
    print("\n" + "=" * 30)
    print("CADASTRAR LIVRO")
    print("=" * 30)

    titulo = input("Título: ").strip()
    autor = input("Autor: ").strip()
    genero = input("Gênero: ").strip().title()


#Verifica se o Livro ja exite

    for livro in biblioteca:
        if livro.titulo.lower() == titulo.lower():
            print(f"❌ Erro: Já existe um livro com o título '{titulo}'.")
            return

    # Validação do ANO
    while True:
        try:
            ano = int(input("Ano de publicação: "))
            if ano <= 0:
                raise ValueError("Ano deve ser positivo.")
            break
        except ValueError:
            print("❌ Erro: Digite um ano válido (número inteiro positivo).")

    # Validação da QUANTIDADE
    while True:
        try:
            copias = int(input("Cópias disponíveis: "))
            if copias < 0:
                raise ValueError("Quantidade não pode ser negativa.")
            break
        except ValueError:
            print("❌ Erro: Digite um número válido de cópias.")

    novo_livro = Livro(titulo, autor, genero, ano, copias)
    biblioteca.append(novo_livro)
    print(f"\n✅ Livro '{titulo}' cadastrado com sucesso!")

# CADASTRAR USUARIOS

def cadastrar_usuario():
    print("\n" + "="*30)
    print("CADASTRAR USUÁRIO")
    print("="*30)

    nome = input("Nome completo: ").strip()
    id = input("ID (número ou matrícula): ").strip()
    contato = input("E-mail ou telefone: ").strip()

    for usuario in usuarios:
        if usuario.id == id:
            print(f"❌ Erro: Já existe um usuário com o ID '{id}'.")
            return

    novo_usuario = Usuario(nome, id, contato)
    usuarios.append(novo_usuario)
    print(f"\n✅ Usuário '{nome}' cadastrado com sucesso!")

# LISTAR LIVROS

def listar_livros():
    print("\n" + "="*30)
    print("LIVROS CADASTRADOS")
    print("="*30)

    if not biblioteca:
        print("Nenhum livro cadastrado.")
    else:
        for i, livro in enumerate(biblioteca, 1):
            print(f"{i}. {livro.mostrar_info()}")

#BUSCAR LIVROS

def buscar_livro():
    print("\n" + "=" * 30)
    print("BUSCA DE LIVROS")
    print("=" * 30)
    termo = input("Digite título, autor, gênero ou ano: ").strip().lower()

    encontrados = []
    for livro in biblioteca:
        # Verifica se o termo está em qualquer atributo (incluindo ano convertido para string)
        if (termo in livro.titulo.lower() or
            termo in livro.autor.lower() or
            termo in livro.genero.lower() or
            termo in str(livro.ano)):  # Novo!
            encontrados.append(livro)

    if encontrados:
        print("\n🔍 Resultados encontrados:")
        for livro in encontrados:
            print(f"- {livro.mostrar_info()}")
    else:
        print("\n❌ Nenhum livro encontrado.")


#EMPRESTAR LIVRO

def emprestar_livro():
    print("\n" + "=" * 30)
    print("EMPRÉSTIMO DE LIVRO")
    print("=" * 30)

    # Passo 1: Pedir dados ao usuário
    id_usuario = input("Digite o ID do usuário: ").strip()
    titulo_livro = input("Digite o título do livro: ").strip()

    # Passo 2: Buscar usuário e livro
    usuario = next((u for u in usuarios if u.id == id_usuario), None)
    livro = next((l for l in biblioteca if l.titulo.lower() == titulo_livro.lower()), None)

    # Passo 3: Validar empréstimo
    if not usuario:
        print("❌ Usuário não encontrado!")
    elif not livro:
        print("❌ Livro não encontrado!")
    elif livro.copias_disponiveis <= 0:
        print("❌ Não há cópias disponíveis deste livro!")
    elif livro.titulo in usuario.livros_emprestados:
        print(f"❌ Erro: {usuario.nome} já possui este livro emprestado.")
    else:
        # Atualiza dados
        livro.copias_disponiveis -= 1
        usuario.livros_emprestados.append(livro.titulo)
        emprestimos.append({
            "id_usuario": usuario.id,
            "titulo_livro": livro.titulo,
            "data": datetime.now().strftime("%d/%m/%Y %H:%M")
        })
        print(f"\n✅ Livro '{livro.titulo}' emprestado para {usuario.nome}!")


#DEVOLVER LIVRO

def devolver_livro():
    print("\n" + "=" * 30)
    print("DEVOLUÇÃO DE LIVRO")
    print("=" * 30)

    # Passo 1: Pedir dados
    id_usuario = input("Digite o ID do usuário: ").strip()
    titulo_livro = input("Digite o título do livro: ").strip()

    # Passo 2: Buscar usuário, livro e empréstimo
    usuario = next((u for u in usuarios if u.id == id_usuario), None)
    livro = next((l for l in biblioteca if l.titulo.lower() == titulo_livro.lower()), None)
    emprestimo = next(
        (e for e in emprestimos
         if e["id_usuario"] == id_usuario and e["titulo_livro"].lower() == titulo_livro.lower()),
        None
    )

    # Passo 3: Validações
    if not usuario:
        print("❌ Usuário não encontrado!")
    elif not livro:
        print("❌ Livro não encontrado!")
    elif not emprestimo:
        print("❌ Este livro não foi emprestado para este usuário!")
    else:
        # Atualiza dados
        livro.copias_disponiveis += 1
        usuario.livros_emprestados.remove(livro.titulo)
        emprestimos.remove(emprestimo)
        print(f"\n✅ Livro '{livro.titulo}' devolvido por {usuario.nome}!")

# RELATORIO DE LIVROS EMPRESTADOS

def relatorio_livros_emprestados():
    print("\n" + "=" * 30)
    print("📊 RELATÓRIO: LIVROS EMPRESTADOS")
    print("=" * 30)

    if not emprestimos:
        print("Nenhum livro emprestado no momento.")
    else:
        print("{:<15} {:<20} {:<25}".format("ID Usuário", "Título do Livro", "Data do Empréstimo"))
        print("-" * 60)
        for emprestimo in emprestimos:
            print("{:<15} {:<20} {:<25}".format(
                emprestimo["id_usuario"],
                emprestimo["titulo_livro"],
                emprestimo.get("data_Emprestimo")
            ))


#Relatorios de Usuario

def relatorio_usuarios():
    print("\n" + "=" * 30)
    print("📋 RELATÓRIO DE USUÁRIOS")
    print("=" * 30)

    if not usuarios:
        print("Nenhum usuário cadastrado.")
    else:
        print("{:<10} {:<20} {:<15}".format("ID", "Nome", "Contato"))
        print("-" * 45)
        for usuario in usuarios:
            print("{:<10} {:<20} {:<15}".format(
                usuario.id,
                usuario.nome,
                usuario.contato
            ))


#Relatorios de estoque

def relatorio_disponibilidade():
    print("\n" + "=" * 30)
    print("📊 STATUS DO ACERVO")
    print("=" * 30)

    total_livros = len(biblioteca)
    total_emprestados = len(emprestimos)
    disponiveis = total_livros - total_emprestados

    print(f"Livros cadastrados: {total_livros}")
    print(f"Livros emprestados: {total_emprestados}")
    print(f"Livros disponíveis: {disponiveis}")


# MENU PRINCIPAL

#CRIAR MENU
# (Todo o código anterior: imports, classes, funções...)

# ========== MENU PRINCIPAL ==========
def main():
    carregar_dados()  # Carrega dados ao iniciar

    while True:
        print("\n" + "=" * 30)
        print("📚 SISTEMA DE BIBLIOTECA")
        print("=" * 30)
        print("1. Cadastrar livro")
        print("2. Cadastrar usuário")
        print("3. Listar livros")
        print("4. Buscar livro")
        print("5. Emprestar livro")
        print("6. Devolver livro")
        print("7. Relatório de empréstimos")
        print("8. Relatório de usuários")
        print("9. Status do moviemntações")
        print("10. Sair")

        opcao = input("Escolha uma opção: ").strip()

        if opcao == "1":
            cadastrar_livro()
        elif opcao == "2":
            cadastrar_usuario()
        elif opcao == "3":
            listar_livros()
        elif opcao == "4":
            buscar_livro()
        elif opcao == "5":
            emprestar_livro()
        elif opcao == "6":
            devolver_livro()
        elif opcao == "7":
            relatorio_livros_emprestados()
        elif opcao == "8":
            relatorio_usuarios()
        elif opcao == "9":
            relatorio_disponibilidade()
        elif opcao == "10":
            salvar_dados()  # Salva dados antes de sair
            print("\nSaindo do sistema... Até logo! 📚")
            break
        else:
            print("\n❌ Opção inválida! Tente novamente.")

# ========== INICIALIZAÇÃO ==========
if __name__ == "__main__":
    main()